In [ ]:
import agentpy as ap
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

class TractorAgent(ap.Agent):
    def setup(self):
        """ Inicialización de variables del tractor """
        self.fuel = 100
        self.efficiency = 0.8
        self.capacity = 10
        self.load = 0
        self.position = (self.model.random.randint(0, 10), self.model.random.randint(0, 10))

    def move(self):
        """ Movimiento hacia una parcela sin cosechar y evita áreas de otros tractores """
        target = self.model.field.find_unharvested_plot(self.position)
        if target:
            self.position = target
            self.load += 1
            self.fuel -= 1

class Field:
    def __init__(self, model, width, height):
        self.model = model
        self.width = width
        self.height = height
        self.grid = [[0 for _ in range(width)] for _ in range(height)]

    def find_unharvested_plot(self, current_position):
        """ Encuentra una parcela sin cosechar que esté disponible """
        for x in range(self.width):
            for y in range(self.height):
                if self.grid[x][y] == 0 and (x, y) != current_position:
                    self.grid[x][y] = 1  # Marca la parcela como cosechada
                    return (x, y)
        return None

class FieldModel(ap.Model):
    def setup(self):
        """ Configuración inicial del campo y de los tractores """
        self.field = Field(self, 10, 10)
        self.agents = ap.AgentList(self, 2, TractorAgent)  # Solo dos tractores
        self.positions = []

    def step(self):
        """ Un paso de simulación donde los tractores se mueven """
        step_positions = []
        for agent in self.agents:
            agent.move()
            step_positions.append(agent.position)
        self.positions.append(step_positions)

# Configuración de la simulación
model = FieldModel()
model.setup()

# Ejecuta la simulación con un límite de pasos
n_steps = 50
for _ in range(n_steps):
    model.step()

# Preparación para la animación
fig, ax = plt.subplots()
ax.set_xlim(0, 9)
ax.set_ylim(0, 9)

# Inicializa un scatter plot para los dos tractores
scat = ax.scatter([], [], c='blue', s=100)  # Inicializa con un solo color

def update(frame):
    """ Actualiza la posición de los tractores en cada frame de la animación """
    positions = model.positions[frame]
    x, y = zip(*positions)
    scat.set_offsets(list(zip(x, y)))
    scat.set_color(['blue', 'green'])  # Actualiza los colores aquí
    return scat,

# Crea la animación
ani = FuncAnimation(fig, update, frames=n_steps, interval=200, blit=True)

# Muestra la animación
plt.xlabel("Eje X (parcela)")
plt.ylabel("Eje Y (parcela)")
plt.title("Simulación del movimiento de dos tractores en la cuadrícula")
plt.show()